In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import itertools
from random import shuffle

In [ ]:
plt.style.use('seaborn-talk')
matplotlib.rcParams.update({'font.size': 15})

In [ ]:
print(plt.style.available)

In [ ]:
# Path to the cleaned data
data_path="C://Users//Dimitri//Desktop//ENSAE3A//Apprentissage en ligne//OnlinePortfolio//TreatedData//DataNyse.csv"
# Location to use to save outputs
output_path = "C://Users//Dimitri//Desktop//ENSAE3A//Apprentissage en ligne//OnlinePortfolio//Outputs//"
# Frequence of rebalancing the portfolio
ndays_rebal = 5
# Eta parameter
eta = 2

In [ ]:
# Load the data
data_nyse = pd.read_csv(data_path, index_col="Date")
# Convert data index to datetime
data_nyse.index = pd.to_datetime(data_nyse.index)
# Print the data for a quick check
print(data_nyse)

In [ ]:
# Get the returns for every ndays_rebal and select only every ndays_rebal-th date
data_returns = data_nyse.pct_change(ndays_rebal).ix[::ndays_rebal, :]
data_returns = data_returns.shift(-1)
data_returns.dropna(how="any", axis=0, inplace=True)
print(data_returns.shape)

## Wealth factor
The following function computes the wealth factor of a strategy determined by a set of weigths $q_{i, t}$. Let $x_{i, t}$ be the returns, $N$ the number of assets and $n$ the number of periods, the wealth factor is given by :
$$ \prod_{t=1}^n \sum_{i=1}^N x_{i, t} q_{i, t} $$

In [ ]:
def wealth_factor(q, x, n):
    """
    Compute the wealth factor for a given strategy q
    
    Args:
        x (pandas.core.frame.DataFrame) : the dataframe of 1 + returns
        q (pandas.core.frame.DataFrame) : the weights for the strategy
        n (int) : number of periods to compute the wealth factor on
    Returns:
        float. The wealth factor for the strategy q
    """
    wealth_coefs = (x * q).sum(axis=1)
    log_wealth_coefs = wealth_coefs.apply(np.log)
    log_wealth_factor = log_wealth_coefs.cumsum()
    return np.exp(log_wealth_factor)

## Internal regret

In [ ]:
def switched_strategy(pvec, i, j):
    """
    Compute the i to j transformed strategy weight vector
    
    Args:
        pvec (pandas.core.series.Series) : the original weight vector
        i (str) : the stock which weight should be allocated to j
        j (str) : the stock keeping its weight and getting i's moreover
    Returns:
        pandas.core.series.Series. The modified weight vector
    """
    switched = pvec.copy()
    switched[j] += switched[i] 
    switched[i] = 0
    return switched

In [ ]:
def no_switch_regret(pvec, xvec, i, j):
    """
    Compute the i to j component to regret for a single vector
    
    Args:
        pvec (pandas.core.series.Series) : the original weight vector
        xvec (pandas.core.series.Series) : the vector of 1 + returns
        i (str) : the stock which weight should be allocated to j
        j (str) : the stock keeping its weight and getting i's moreover
    Returns:
        float. The regret component
    """
    switched = switched_strategy(pvec, i, j)
    return np.log((switched * xvec).sum()) -  np.log((pvec * xvec).sum())

In [ ]:
def pairs_of_stocks(p):
    stocks_pairs = list(itertools.product(p.columns, p.columns))
    stocks_pairs_str = []
    for pair in stocks_pairs:
        if pair[0] != pair[1]:
            stocks_pairs_str.append(pair[0] + "/" + pair[1])
    return stocks_pairs_str

In [ ]:
def no_switch_regret_tab(p, x):
    stocks_pairs_str = pairs_of_stocks(p)
    regret_tab = pd.DataFrame(index=p.index, columns=stocks_pairs_str)
    for t in p.index:
        for pair in stocks_pairs_str :
            stock1, stock2 = pair.split("/")[0], pair.split("/")[1]
            regret_tab.loc[t, pair] = no_switch_regret(p.loc[t, :], x.loc[t, :], stock1, stock2)
    return regret_tab

In [ ]:
def internal_regret_tab(p, x):
    r_tab = no_switch_regret_tab(p, x).cumsum().max(axis=1)
    return r_tab

## Exp weights coefs
For a given set of weights at time $t-1$, will compute the vector of actualization of the weights at time $t$ which is given by the components :
$$ \frac{p_{i, t} \exp \left( \frac{\eta x_{i, t}} {p_t x_t} \right)}{\sum_{j=1}^N p_{j, t} \exp \left( \frac{\eta x_{j, t}} {p_t x_t} \right)} $$

In [ ]:
def exp_weights(pvec, xvec, eta):
    """
    Computes the vector of actualization of the weights for the EG strategy
    
    Args:
        pvec (pandas.core.series.Series) : the vector of previous weights
        xvec (pandas.core.series.Series) : the vector of previous 1 + returns
        eta (float) : eta parameter
    Returns:
        pandas.core.series.Series. The vector of actualization of weights for the EG strategy
    """
    norm_coef = (pvec * xvec).sum()
    wvec = pd.Series(index=pvec.index)
    for i in pvec.index:
        wvec[i] = pvec[i] * np.exp (eta * xvec[i] / norm_coef)
    return wvec / wvec.sum()

In [ ]:
def weights_eg(xtab, eta, pinit):
    """
    Compute the weights for the EG strategy
    
    Args:
        xtab (pandas.core.frame.DataFrame) : the data frame of returns
        eta (float) : the eta parameter
        pinit (pandas.core.series.Series) : the series of initial weights, we must have pinit.index = xtab.columns
    """
    wtab = pd.DataFrame(index=xtab.index, columns=xtab.columns)
    for t in range(0, wtab.shape[0]):
        if t==0:
            wtab.iloc[t, :] = pinit
        else :
            coefs = exp_weights(wtab.iloc[t-1, :], xtab.iloc[t-1, :], eta)
            wtab.iloc[t, :] = coefs
    return wtab

In [ ]:
# Test for weights_eg
x = 1 + data_returns.iloc[:, 10:15]
pinit = pd.Series(index=x.columns, data=1/len(x.columns))
w = weights_eg(x, 1, pinit)
fig, ax = plt.subplots(1, 1, figsize=(10.0, 6.5))
w.plot(ax=ax)
plt.title("EG weights example - monthly rebalancing")
plt.ylabel("weight")
fig.savefig(output_path + "EG_weights_example.svg")
plt.show()

In [ ]:
# Test for wealth factor on the EG strategy
w_fact = wealth_factor(x, w, 10)
w_fact.plot()
plt.show()

In [ ]:
# Test for internal regret
ir = internal_regret_tab(w, x)
ir.plot()
plt.show()

## Small external regret to small internal regret conversion

In [ ]:
def induced_matrix(delta):
    """
    Generate the A matrix from the article from a distribution on the pairs of stocks
    
    Args:
        delta (pandas.core.series.Series) : a distribution on the pairs of stocks
    Returns:
        pandas.core.frame.DataFrame :  the A matrix as a pandas DataFrame
    """
    stocks_list = []
    for pair in delta.index:
        stocks_list.append(pair.split("/")[0])
        stocks_list.append(pair.split("/")[1])
    stocks = np.unique(stocks_list)  
    A = pd.DataFrame(index=stocks, columns=stocks)
    for m in stocks:
        for i in stocks:
            if i != m:
                A.loc[m, i] = delta.loc[m + "/" + i]
    for m in stocks:
        A.loc[m, m] = - A.loc[:, m].sum()
    return A

In [ ]:
def get_kernel_index(eig_vals):
    """
    From a vector of complex eigen values, find the indices of those corresponding to 0 and return their index
    
    Args:
        eig_vals (numpy.ndarray) : the vector of complex  eigen values
    Returns:
        numpy.ndarray. The array of indexes for the null eigen values
    """
    eig_vals_im, eig_vals_real = np.imag(eig_vals), np.real(eig_vals)
    eig_vals_im.setflags(write=1)
    eig_vals_real.setflags(write=1)
    eig_vals_im[abs(eig_vals_im) < 1e-6] = 0
    eig_vals_real[abs(eig_vals_real) < 1e-6] = 0
    ind_im = np.argwhere(eig_vals_im == 0)
    ind_real = np.argwhere(eig_vals_real == 0)
    null_ind = np.intersect1d(ind_im, ind_real)
    return null_ind

In [ ]:
def choose_proba_distrib(vecs):
    """
    Choose a probability distribution among a set of kernel eigen vectors
    
    Args:
        vecs (numpy.ndarray) : the kernel generating vectors stacked in columns
    Returns:
        numpy.ndarray : the selected kernel vector
    """
    candidates = []
    for i in range(0, vecs.shape[1]):
        if np.allclose(np.imag(vecs[:, i]), 0):
            if np.all(vecs[:, i] >= 0):
                candidates.append(np.real(vecs[:, i]))
            elif np.all(vecs[:, i] <= 0):
                candidates.append(-np.real(vecs[:, i]))       
    if len(candidates) >= 1:
        return candidates[0] / candidates[0].sum()
    else :
        print("No probability distribution in eigen vector")
        raise np.linalg.LinAlgError

In [ ]:
def fixed_point_proba(A):
    """
    Find a probability vector in the kernel of A if it exists
    
    Args:
        A (pandas.core.frame.DataFrame) : the A matrix
    Returns:
        pandas.core.series.Series. The kernel probability distribution
    """
    for col in A.columns:
        A.loc[:, col] = pd.to_numeric(A.loc[:, col])
    Amat = A.as_matrix()
    eig_vals, eig_vecs = np.linalg.eig(Amat)
    null_ind = get_kernel_index(eig_vals)
    kernel_vecs = eig_vecs[:, null_ind]
    kernel_vec_pd = pd.Series(index=A.columns, data=choose_proba_distrib(kernel_vecs))
    return kernel_vec_pd

In [ ]:
def induced_portfolio(delta):
    """
    Induce portfolio on the assets from proba distribution on the pairs of assets
    
    Args:
        delta (pandas.core.series.Series) : a distribution on the pairs of stocks
    Returns:
        pandas.core.series.Series. the induced probability distribution on the stocks
    """
    A = induced_matrix(delta)
    return fixed_point_proba(A)

## BP1EXP and BP1POL

In [ ]:
def delta_init(xtab) : 
    pairs_index = pairs_of_stocks(xtab)
    delta = pd.Series(data=1 / len(pairs_index), index=pairs_index)
    return delta 

In [ ]:
def delta_update_exp(delta_prev, pvec, xvec, eta):
    norm_coef = 0
    delta = pd.Series(index=delta_prev.index)
    for pair in delta_prev.index:
        i, j = pair.split("/")[0], pair.split("/")[1]
        p_dot_x = (pvec * xvec).sum()
        num = np.exp(- eta * pvec.loc[i] * ((xvec[j] / p_dot_x) - (xvec[i] / p_dot_x)))
        delta.loc[pair] = delta_prev.loc[pair] * num
        norm_coef += delta_prev.loc[pair] * num
    delta *= 1 / norm_coef
    return delta

In [ ]:
# Test for delta_update_exp
delta0 = delta_init(x)
delta1 = delta_update_exp(delta0, w.iloc[0, :], x.iloc[0, :], 2)
print(delta1)

In [ ]:
# Test for induced_matrix
amat = induced_matrix(delta1)
print(amat)

In [ ]:
# Test for induced_portfolio
induced = induced_portfolio(delta1)
print(induced)

In [ ]:
def delta_exp(xtab, eta):
    delta0 = delta_init(xtab)
    delta_tab = pd.DataFrame(index=xtab.index, columns=delta0.index)
    ptab = pd.DataFrame(index=xtab.index, columns=xtab.columns)
    delta_tab.iloc[0, :] = delta0
    ptab.iloc[0, :] = induced_portfolio(delta_tab.iloc[0, :])
    for t in range(1, xtab.shape[0]):
        delta_tab.iloc[t, :] = delta_update_exp(delta_tab.iloc[t-1, :], ptab.iloc[t-1, :], xtab.iloc[t-1, :], eta)
        ptab.iloc[t, :] = induced_portfolio(delta_tab.iloc[t, :])
    return delta_tab, ptab

In [ ]:
# Choice of stocks
x = 1 + data_returns.iloc[:200, 50:55]

In [ ]:
# Test for delta_exp
eta1 = 2
delta, p_bp1exp = delta_exp(x, eta1)

In [ ]:
# Comparison between EG and BP1EXP
pinit = pd.Series(index=x.columns, data=1/len(x.columns))
p_eg = weights_eg(x, eta1, pinit)

# Comparison of weights
fig1, axes = plt.subplots(3, 1, figsize=(10.0, 13.0), sharex=True)
p_eg.plot(ax=axes[0])
axes[0].set_title("EG weights")
p_bp1exp.plot(ax=axes[1])
axes[1].set_title("BP1EXP weights")
(p_eg - p_bp1exp).plot(ax=axes[2])
axes[2].set_title("EG - BP1EXP")
fig1.savefig(output_path + "Weights_example.svg")
plt.show()

#Comparison of wealth factor
plt.figure()
wfact_eg = wealth_factor(p_eg, x, 10)
wfact_bp1exp = wealth_factor(p_bp1exp, x, 10)
wfact_bp1exp.plot(label="BP1EXP")
wfact_eg.plot(label="EG")
plt.legend()
plt.title("Wealth factor comparison")
plt.show()

# Comparison of internal regret
plt.figure()
ir_eg = internal_regret_tab(p_eg, x)
ir_bp1exp = internal_regret_tab(p_bp1exp, x)
ir_bp1exp.plot(label="BP1EXP")
ir_eg.plot(label="EG")
plt.legend()
plt.title("Internal regret comparison")
plt.show()

## Robustness evalution by Monte Carlo

In [ ]:
def stock_random_draws(returns, nstocks, ndraws):
    """
    Draw a sequence of randomly chosen sets of stocks
    
    Args:
        returns (pandas.core.frame.DataFrame) : the database of returns
        nstocks (int) : number of stock for each draws
        ndraws (int) : number of draws to perform
    Returns:
        pandas.core.frame.DataFrame. The random drawn stocks stacked in a dataframe
    """
    stocks_draws = pd.DataFrame(index=range(0, ndraws), columns=range(0, nstocks))
    for draw in range(0, ndraws):
        stock_inds = list(range(0, len(returns.columns)))
        shuffle(stock_inds)
        chosen = stock_inds[0:nstocks]
        stocks_draws.loc[draw, :] = chosen
    return stocks_draws

In [ ]:
def bp1exp_mc_robustness(returns, random_draws, eta, nperiods):
    """
    Average over randomly picked sets of stocks for internal regrets and wealth factor for BP1EXP strategy
    
    Args:
        returns (pandas.core.frame.DataFrame) : the database of returns
        random_draws (pandas.core.frame.DataFrame): the data frame of random draws
        eta (float) : the eta parameter
        nperiods(int) : the number of periods to consider
    Returns:
        tuple. A tuple of pandas.core.series.Series (average_wealth_factor_series, average_internal_regret_series)
    """
    int_reg_sum_bp1exp = pd.Series(data=0, index=returns.index[:nperiods])
    wfact_sum_bp1exp = pd.Series(data=0, index=returns.index[:nperiods])
    for it in range(0, random_draws.shape[0]):
        x = 1 + returns.iloc[:nperiods, random_draws.loc[it, :]]
        delta, p_bp1exp = delta_exp(x, eta)
        wfact_bp1exp = wealth_factor(p_bp1exp, x, 10)
        ir_bp1exp = internal_regret_tab(p_bp1exp, x)
        wfact_sum_bp1exp += wfact_bp1exp
        int_reg_sum_bp1exp += ir_bp1exp
        print(it)
    return (1 / random_draws.shape[0]) * int_reg_sum_bp1exp, (1 / random_draws.shape[0]) * wfact_sum_bp1exp

In [ ]:
def eg_mc_robustness(returns, random_draws, eta, nperiods):
    """
    Average over randomly picked sets of stocks for internal regrets and wealth factor for EG strategy
    
    Args:
        returns (pandas.core.frame.DataFrame) : the database of returns
        random_draws (pandas.core.frame.DataFrame): the data frame of random draws
        eta (float) : the eta parameter
        nperiods(int) : the number of periods to consider
    Returns:
        tuple. A tuple of pandas.core.series.Series (average_wealth_factor_series, average_internal_regret_series)
    """
    int_reg_sum_eg = pd.Series(data=0, index=returns.index[:nperiods])
    wfact_sum_eg = pd.Series(data=0, index=returns.index[:nperiods])
    for it in range(0, random_draws.shape[0]):
        x = 1 + returns.iloc[:nperiods, random_draws.loc[it, :]]
        pinit = pd.Series(index=x.columns, data=1/len(x.columns))
        p_eg = weights_eg(x, eta, pinit)
        wfact_eg = wealth_factor(p_eg, x, 10)
        ir_eg = internal_regret_tab(p_eg, x)
        wfact_sum_eg += wfact_eg 
        int_reg_sum_eg += ir_eg
        print(it)
    return (1 / random_draws.shape[0]) * int_reg_sum_eg, (1 / random_draws.shape[0]) * wfact_sum_eg

In [ ]:
def compare_several_etas (returns, random_draws, etas_eg, etas_bp1exp, nperiods):
    results_list = []
    for i in range(0, len(etas_eg)):
        ir_eg, wf_eg = eg_mc_robustness(returns, random_draws, etas_eg[i], nperiods)
        ir_bp1exp, wf_bp1exp = bp1exp_mc_robustness(returns, random_draws, etas_bp1exp[i], nperiods)
        results_list.append((ir_eg, wf_eg, ir_bp1exp, wf_bp1exp))
    return results_list

In [ ]:
# MC averaging parameters
nstocks = 10
nperiods = data_returns.shape[0]
nit = 200
# Choice of eta to minimize the internal regret bound in the theory
alpha = 0.3
eta_opti_eg = alpha * np.sqrt(8 * np.log(nstocks) / nperiods)
eta_opti_bp1exp = 4 * alpha * np.sqrt(np.log(nstocks) / nperiods)
print(eta_opti_bp1exp)
print(eta_opti_eg)
# Etas to compare
etas_eg = [eta_opti_eg, 0.5, 1, 2]
etas_bp1exp = [eta_opti_bp1exp, 0.5, 1, 2]

In [ ]:
# MC computations
stocks_draws = stock_random_draws(data_returns, nstocks, nit)
results = compare_several_etas(data_returns, stocks_draws, etas_eg, etas_bp1exp, nperiods)

In [ ]:
def eg_vs_bp1exp_plot(etas, results, nstocks, ndraws, nperiods, save_file="EG_vs_BP1EXP_Bis.svg", opti_ind=0):
    fig, axes = plt.subplots(len(etas), 2, figsize=(10.0, 13.0), sharex=True)
    plt.suptitle("EG vs BP1EXP - Monthly rebalancing - averaged over " + str(nit) + " choices of " + str(nstocks) + " stocks")
    count = 0
    for eta in etas:
        if count == opti_ind:
            axes[count, 0].set_title("Wealth factor - optimal eta")
            axes[count, 1].set_title("Internal regret - optimal eta")
        else:
            axes[count, 0].set_title("Wealth factor - eta=" + str(eta))
            axes[count, 1].set_title("Internal regret - eta=" + str(eta))
        results[count][1].plot(ax=axes[count, 0], label="EG")
        results[count][3].plot(ax=axes[count, 0], label="BP1EXP")
        results[count][0].plot(ax=axes[count, 1], label="EG")
        results[count][2].plot(ax=axes[count, 1], label="BP1EXP")
        axes[count, 0].legend()
        axes[count, 1].legend()
        count += 1
    fig.savefig(output_path + save_file)
    plt.show()

In [ ]:
eg_vs_bp1exp_plot(etas_eg, results, nstocks, nit, nperiods)